In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from src.imagenet_x import get_factor_accuracies, error_ratio
from src.imagenet_x.utils import load_model_predictions, get_annotation_path
from src.imagenet_x import plots
from src.imagenet_x.evaluate import ImageNetX, get_vanilla_transform, ImageNetXImageFolder, load_annotations
import os
import pickle
from pathlib import Path

In [ ]:
df1 = pd.read_csv('../metalabel_objectivity/val_imgs_df.csv')

root = "../data/ImageNetVal/"

l1 = {}
for path, subdirs, files in os.walk(root):
    for name in files:
        l1[name] = os.path.join(path, name)

df1['path'] = df1['file_name'].apply(lambda x: l1[x])


samples = df1.sample(100, random_state = 420)

samplescols = samples.columns[1:17]
k = samples.iloc[:, 1:17].values
samples_labels = []
l1 = []
for item in k:
    l1 = []
    for i, thing in enumerate(item):
        if thing == 1:
            l1.append(samplescols[i])
    samples_labels.append(l1)

samples['labels'] = samples_labels

proto_df = pd.read_csv('../data/annotations/prototypical_paths.csv')
proto_df = proto_df.merge(df1, on='file_name', how='inner')

def plot_samples(which_sample):
    samples_proto = proto_df[proto_df['label'] == samples.iloc[which_sample]['label']]
    #  plot 6 in a grid with matplotlib cropping them
    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    img = plt.imread(samples.iloc[which_sample]['path'])
    ax[0].imshow(img)
    ax[0].set_title(samples.iloc[which_sample]['labels'], fontsize=22)
    props = dict(boxstyle='round', facecolor='cyan', alpha=0.6)
    ax[0].text(0.05, 0.95, samples.iloc[which_sample]['str_label'], transform=ax[0].transAxes, fontsize=22,
        verticalalignment='top', bbox=props)
    ax[0].axis('off')

    for i,row in enumerate(samples_proto.iterrows()):
        img = plt.imread(row[1]['path'])
        ax[i+1].imshow(img)
        ax[i+1].axis('off')
    fig.tight_layout()

for i in range(len(samples)):
    plot_samples(i)